In [18]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import albumentations as A
from albumentations.pytorch import ToTensorV2
from PIL import Image
import numpy as np
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report
import io

In [19]:
import mlflow
import mlflow.pytorch

In [20]:
mlflow.set_experiment("MLP_Clasificador_Imagenes")

<Experiment: artifact_location='file:///Users/majotagliaferro/Documents/TPRN/skin-dataset-classification-main/mlruns/249710420298442039', creation_time=1763846127725, experiment_id='249710420298442039', last_update_time=1763846127725, lifecycle_stage='active', name='MLP_Clasificador_Imagenes', tags={'mlflow.experimentKind': 'custom_model_development'}>

In [21]:
from torch.utils.tensorboard import SummaryWriter
import torchvision.utils as vutils

In [22]:
# Función para loguear una figura matplotlib en TensorBoard
def plot_to_tensorboard(fig, writer, tag, step):
    buf = io.BytesIO()
    fig.savefig(buf, format='png')
    buf.seek(0)
    image = Image.open(buf).convert("RGB")
    image = np.array(image)
    image = torch.tensor(image).permute(2, 0, 1) / 255.0
    writer.add_image(tag, image, global_step=step)
    plt.close(fig)

In [23]:
# Función para matriz de confusión y clasificación
def log_classification_report(model, loader, writer, step, prefix="val"):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, labels in loader:
            images = images.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.numpy())

    # Confusion matrix
    cm = confusion_matrix(all_labels, all_preds)
    fig_cm, ax = plt.subplots(figsize=(6, 6))
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=train_dataset.label_encoder.classes_)
    disp.plot(ax=ax, cmap='Blues', xticks_rotation=45)
    ax.set_title(f'{prefix.title()} - Confusion Matrix')

    # Guardar localmente y subir a MLflow
    fig_path = f"confusion_matrix_{prefix}_epoch_{step}.png"
    fig_cm.savefig(fig_path)
    mlflow.log_artifact(fig_path)
    os.remove(fig_path)

    plot_to_tensorboard(fig_cm, writer, f"{prefix}/confusion_matrix", step)

    cls_report = classification_report(all_labels, all_preds, target_names=train_dataset.label_encoder.classes_)
    writer.add_text(f"{prefix}/classification_report", f"<pre>{cls_report}</pre>", step)

    # También loguear texto del reporte
    with open(f"classification_report_{prefix}_epoch_{step}.txt", "w") as f:
        f.write(cls_report)
    mlflow.log_artifact(f.name)
    os.remove(f.name)


In [24]:

# Crear directorio de logs
log_dir = "runs/mlp_experimento_1"
writer = SummaryWriter(log_dir=log_dir)

In [25]:
class CustomImageDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform

        self.image_paths = []
        self.labels = []

          # SOLO carpetas válidas
        class_names = sorted([
            d for d in os.listdir(root_dir)
            if os.path.isdir(os.path.join(root_dir, d))
        ])

        self.class_to_idx = {cls: idx for idx, cls in enumerate(class_names)}

        for cls in class_names:
            cls_dir = os.path.join(root_dir, cls)

            # Evitar errores si la carpeta está corrupta
            try:
                files = os.listdir(cls_dir)
            except NotADirectoryError:
                continue

            for fname in files:
                if fname.lower().endswith((".png", ".jpg", ".jpeg")):
                    self.image_paths.append(os.path.join(cls_dir, fname))
                    self.labels.append(cls)

        # Encode labels
        self.label_encoder = LabelEncoder()
        self.labels = self.label_encoder.fit_transform(self.labels)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = np.array(Image.open(self.image_paths[idx]).convert("RGB"))
        label = self.labels[idx]

        if self.transform:
            augmented = self.transform(image=image)
            image = augmented["image"]

        return image, label

In [26]:
train_transform = A.Compose([
    A.Resize(64, 64),
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.2),
    A.Normalize(),
    ToTensorV2()
])


In [27]:
val_test_transform = A.Compose([
    A.Resize(64, 64),
    A.Normalize(),
    ToTensorV2()
])

In [28]:
# Paths
train_dir = "data/Split_smol/train"
val_dir = "data/Split_smol/val/"

In [29]:
train_dataset = CustomImageDataset(train_dir, transform=train_transform)
val_dataset   = CustomImageDataset(val_dir, transform=val_test_transform)

batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=batch_size)

In [65]:
class MLPClassifier(nn.Module):
    def __init__(self, input_size=64*64*3, num_classes=10):
        super().__init__()
        self.model = nn.Sequential(nn.Flatten(),
            nn.Linear(input_size, 512),
            nn.ReLU(),
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Linear(128, num_classes))
        
    def init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.uniform_(m.weight)
                nn.init.zeros_(m.bias)        
    def forward(self, x):
        return self.model(x)

In [66]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_classes = len(set(train_dataset.labels))
model = MLPClassifier(num_classes=num_classes).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [67]:
# Entrenamiento y validación
def evaluate(model, loader, epoch=None, prefix="val"):
    log_classification_report(model, val_loader, writer, step=epoch, prefix="val")
    model.eval()
    correct, total, loss_sum = 0, 0, 0.0

    all_preds = []
    all_labels = []

    with torch.no_grad():
        for i, (images, labels) in enumerate(loader):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            _, preds = torch.max(outputs, 1)

            loss_sum += loss.item()
            correct += (preds == labels).sum().item()
            total += labels.size(0)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

            # Loguear imágenes del primer batch
            if i == 0 and epoch is not None:
                img_grid = vutils.make_grid(images[:8].cpu(), normalize=True)
                writer.add_image(f"{prefix}/images", img_grid, global_step=epoch)

    acc = 100.0 * correct / total
    avg_loss = loss_sum / len(loader)

    if epoch is not None:
        writer.add_scalar(f"{prefix}/loss", avg_loss, epoch)
        writer.add_scalar(f"{prefix}/accuracy", acc, epoch)

    return avg_loss, acc

In [ ]:
# # Loop de entrenamiento
# n_epochs = 10
# with mlflow.start_run():
#     # Log hiperparámetros
#     mlflow.log_params({
#         "model": "MLPClassifier",
#         "input_size": 64*64*3,
#         "batch_size": batch_size,
#         "lr": 1e-3,
#         "epochs": n_epochs,
#         "optimizer": "Adam",
#         "loss_fn": "CrossEntropyLoss",
#         "train_dir": train_dir,
#         "val_dir": val_dir,
#     })
#     for epoch in range(n_epochs):
#         model.train()
#         running_loss = 0.0
#         correct, total = 0, 0
    
#         for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{n_epochs}"):
#             images, labels = images.to(device), labels.to(device)
    
#             optimizer.zero_grad()
#             outputs = model(images)
#             loss = criterion(outputs, labels)
#             loss.backward()
#             optimizer.step()
    
#             running_loss += loss.item()
#             _, preds = torch.max(outputs, 1)
#             correct += (preds == labels).sum().item()
#             total += labels.size(0)
   
#         for name, param in model.named_parameters():
#             writer.add_histogram(name, param, epoch)
    
#         train_loss = running_loss / len(train_loader)
#         train_acc = 100.0 * correct / total
#         val_loss, val_acc = evaluate(model, val_loader, epoch=epoch, prefix="val")
    
#         print(f"Epoch {epoch+1}:")
#         print(f"  Train Loss: {train_loss:.4f}, Accuracy: {train_acc:.2f}%")
#         print(f"  Val   Loss: {val_loss:.4f}, Accuracy: {val_acc:.2f}%")
    
#         writer.add_scalar("train/loss", train_loss, epoch)
#         writer.add_scalar("train/accuracy", train_acc, epoch)
    
#         # Log en MLflow
#         mlflow.log_metrics({
#             "train_loss": train_loss,
#             "train_accuracy": train_acc,
#             "val_loss": val_loss,
#             "val_accuracy": val_acc
#         }, step=epoch)
#         # Guardar modelo
#     torch.save(model.state_dict(), "mlp_model.pth")
#     print("Modelo guardado como 'mlp_model.pth'")
#     mlflow.log_artifact("mlp_model.pth")
#     mlflow.pytorch.log_model(model, artifact_path="pytorch_model")
#     print("Modelo guardado como 'mlp_model.pth'")

Epoch 1/10: 100%|██████████| 22/22 [00:15<00:00,  1.46it/s]
/opt/miniconda3/envs/{tpredes}/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/opt/miniconda3/envs/{tpredes}/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/opt/miniconda3/envs/{tpredes}/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this beha

Epoch 1:
  Train Loss: 2.6753, Accuracy: 27.55%
  Val   Loss: 1.5801, Accuracy: 41.99%


Epoch 2/10: 100%|██████████| 22/22 [00:11<00:00,  1.88it/s]


Epoch 2:
  Train Loss: 1.5808, Accuracy: 43.76%
  Val   Loss: 1.5909, Accuracy: 48.07%


Epoch 3/10: 100%|██████████| 22/22 [00:11<00:00,  1.88it/s]
/opt/miniconda3/envs/{tpredes}/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/opt/miniconda3/envs/{tpredes}/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/opt/miniconda3/envs/{tpredes}/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this beha

...

Epoch 9:
  Train Loss: 0.9035, Accuracy: 66.43%
  Val   Loss: 1.1960, Accuracy: 65.19%


Epoch 10/10: 100%|██████████| 22/22 [00:11<00:00,  1.91it/s]
2025/11/24 19:19:42 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


Epoch 10:
  Train Loss: 0.8634, Accuracy: 65.28%
  Val   Loss: 1.2696, Accuracy: 55.25%
Modelo guardado como 'mlp_model.pth'


2025/11/24 19:19:52 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Modelo guardado como 'mlp_model.pth'


In [ ]:
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime
import mlflow
import torch
from tqdm import tqdm

# -----------------------------------------------------------
# CREAR NOMBRE ÚNICO PARA EL RUN DE TENSORBOARD
# -----------------------------------------------------------
run_name = f"mlp_experimento_1_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
writer = SummaryWriter(f"runs/{run_name}")

print(f"📌 TensorBoard run directory: runs/{run_name}")

# -----------------------------------------------------------
# LOOP DE ENTRENAMIENTO
# -----------------------------------------------------------
n_epochs = 10

with mlflow.start_run():

    # Log hiperparámetros
    mlflow.log_params({
        "model": "MLPClassifier",
        "input_size": 64*64*3,
        "batch_size": batch_size,
        "lr": 1e-3,
        "epochs": n_epochs,
        "optimizer": "Adam",
        "loss_fn": "CrossEntropyLoss",
        "train_dir": train_dir,
        "val_dir": val_dir,
        "tensorboard_run": run_name
    })

    for epoch in range(n_epochs):

        model.train()
        running_loss = 0.0
        correct, total = 0, 0

        for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{n_epochs}"):

            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

        # Histograma de parámetros
        for name, param in model.named_parameters():
            writer.add_histogram(f"weights/{name}", param, epoch)

        # Métricas
        train_loss = running_loss / len(train_loader)
        train_acc = 100.0 * correct / total

        # Val
        val_loss, val_acc = evaluate(model, val_loader, epoch=epoch, prefix="val")

        print(f"Epoch {epoch+1}:")
        print(f"  Train Loss: {train_loss:.4f}, Accuracy: {train_acc:.2f}%")
        print(f"  Val   Loss: {val_loss:.4f}, Accuracy: {val_acc:.2f}%")

        # Log a TensorBoard
        writer.add_scalar("train/loss", train_loss, epoch)
        writer.add_scalar("train/accuracy", train_acc, epoch)
        writer.add_scalar("val/loss", val_loss, epoch)
        writer.add_scalar("val/accuracy", val_acc, epoch)

        # Log a MLflow
        mlflow.log_metrics({
            "train_loss": train_loss,
            "train_accuracy": train_acc,
            "val_loss": val_loss,
            "val_accuracy": val_acc
        }, step=epoch)

    # -------------------------------------------------------
    # GUARDAR MODELO
    # -------------------------------------------------------
    torch.save(model.state_dict(), "mlp_model.pth")
    print("Modelo guardado como 'mlp_model.pth'")

    mlflow.log_artifact("mlp_model.pth")
    mlflow.pytorch.log_model(model, artifact_path="pytorch_model")

print("🏁 Entrenamiento finalizado correctamente.")

📌 TensorBoard run directory: runs/mlp_experimento_1_20251124_193642


Epoch 1/10: 100%|██████████| 22/22 [00:15<00:00,  1.44it/s]
/opt/miniconda3/envs/{tpredes}/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/opt/miniconda3/envs/{tpredes}/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/opt/miniconda3/envs/{tpredes}/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this beha

Epoch 1:
  Train Loss: 2.7122, Accuracy: 30.13%
  Val   Loss: 1.9659, Accuracy: 34.25%


Epoch 2/10: 100%|██████████| 22/22 [00:18<00:00,  1.22it/s]
/opt/miniconda3/envs/{tpredes}/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/opt/miniconda3/envs/{tpredes}/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/opt/miniconda3/envs/{tpredes}/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this beha

Epoch 2:
  Train Loss: 1.6931, Accuracy: 44.19%
  Val   Loss: 1.5363, Accuracy: 49.17%


...

Epoch 9:
  Train Loss: 1.0285, Accuracy: 60.98%
  Val   Loss: 1.5037, Accuracy: 56.35%


Epoch 10/10: 100%|██████████| 22/22 [00:11<00:00,  1.92it/s]
2025/11/24 19:39:53 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


Epoch 10:
  Train Loss: 0.9587, Accuracy: 64.28%
  Val   Loss: 1.4074, Accuracy: 51.93%
Modelo guardado como 'mlp_model.pth'


2025/11/24 19:40:03 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏁 Entrenamiento finalizado correctamente.


In [55]:
!tensorboard --logdir=runs/mlp_experimento_1

/opt/miniconda3/envs/{tpredes}/lib/python3.12/site-packages/tensorboard/default.py:30: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
TensorFlow installation not found - running with reduced feature set.

NOTE: Using experimental fast data loading logic. To disable, pass
    "--load_fast=false" and report issues on GitHub. More details:
    https://github.com/tensorflow/tensorboard/issues/4784

Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.20.0 at http://localhost:6006/ (Press CTRL+C to quit)
^C
